In [ ]:
import numpy as np
from pandas import DataFrame
import keras

In [ ]:
import matplotlib.pyplot as plt
#%matplotlib notebook

# Récupération et mise en forme des données

In [ ]:
X = np.load('X.npy')
Y = np.load('Y.npy')

In [ ]:
X.shape

In [ ]:
X_flat = np.array(X).reshape((2062, 64*64))
X_flat.shape

In [ ]:
Ydf = DataFrame(data=Y, columns=[9-x for x in range(10)])
Ydf.head()

In [ ]:
Ydf = Ydf[[x for x in range(10)]]
Ydf.head()

In [ ]:
Ydf['class']=(Ydf[range(10)].idxmax(axis=1)).astype('int')
Ydf[['class']]

In [ ]:
Y_flat = Ydf['class'].to_numpy()
Y_flat.shape

In [ ]:
Y_flat = Y_flat.reshape(-1,1)
Y_flat.shape

In [ ]:
from numpy import random as np_random

SEED = 42
np_random.seed(SEED)

In [ ]:
# LEARNING_RATE = 0.001
# EPOCHS = 500

# Model Keras Application :

In [ ]:
from keras.applications import *
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout #create model
from keras.optimizers import Adam, Nadam
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import pandas as pd
from sklearn.model_selection import train_test_split

## Transfert learning avec predict

In [ ]:
X_prepared_tl = np.stack((X,X,X), axis=3)
X_prepared_tl.shape

In [ ]:
def tl_features(app_keras, inputs):
    model = app_keras(include_top=False, 
                            weights='imagenet', 
                            input_shape=(inputs.shape[1],inputs.shape[2],inputs.shape[3]), 
                            pooling=max)
    features = model.predict(inputs)
    features = features.reshape(features.shape[0],
                                (features.shape[1]*features.shape[2]*features.shape[3]))
    return features

In [ ]:
features_vgg16 = tl_features(VGG16, X_prepared_tl)
print(features_vgg16.shape)

In [ ]:
features_mobilenet = tl_features(MobileNet, X_prepared_tl)
print(features_mobilenet.shape)

In [ ]:
features_vgg19 = tl_features(VGG19, X_prepared_tl)
print(features_vgg19.shape)

In [ ]:
def tl_model(features, output, OPTIMIZER, LEARNING_RATE, EPOCHS, BATCH_SIZE, nb_dense, ACTIVATION, INITIALIZER):
    X_train, X_val, y_train, y_val = train_test_split(features, output, test_size=0.1)
    
    model = Sequential() #add model layers
    model.add(Dense(32, activation=ACTIVATION, kernel_initializer=INITIALIZER))
    model.add(Dropout(0.3))
    model.add(Dense(16, activation=ACTIVATION, kernel_initializer=INITIALIZER))
    model.add(Dropout(0.3))
    model.add(Dense(nb_dense, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', 
                    optimizer=OPTIMIZER(lr=LEARNING_RATE), 
                    metrics = ['accuracy'])
    
    history = model.fit(X_train,
                        y_train,
                        validation_data=(X_val, y_val),
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        verbose=2)
    
    pd.DataFrame(history.history).plot()
    plt.grid(True)
    plt.title('Model performance throughout training CNN')
    plt.ylabel('metrics')
    plt.xlabel('epoch')
    plt.legend(['val_loss', 'val_acc', 'train_loss', 'train_acc'], loc='upper right')
    plt.show()
    return model

In [ ]:
kwargs = {
    'features' : features_vgg16,
    'output' : Y_flat,
    'OPTIMIZER' : Nadam,
    'LEARNING_RATE' : 0.001,
    'EPOCHS' : 50,
    'BATCH_SIZE' : 64,
    'nb_dense' : 10,
    'ACTIVATION' : 'elu',
    'INITIALIZER' : 'normal'
}

In [ ]:
model_vgg16 = tl_model(**kwargs)

In [ ]:
model_vgg19 = tl_model(**kwargs)

In [ ]:
model_mobilenet = tl_model(**kwargs)